## folium 개요

folium: `leaflet.js` 기반으로 만들어진 Python 지도 시각화 라이브러리

folium을 사용하여 인터랙티브한 지도를 생성하고 **마커를 추가**하여 시각화하거나 **원으로 범위를 표기**하고 `html` 파일로 내보내기 등을 수행할 수 있다.

## folium 설치

In [ ]:
!pip install folium

모듈 import 

In [ ]:
import folium

## 기본 좌표 설정

`location`에 입력한 **위,경도 좌표**를 기준으로 지도를 그릴 수 있다.

`zoom_start`의 범위는 **최대 18** 

In [ ]:
# 위도
latitude = 37.555946

# 경도
longitude = 126.972317

In [ ]:
# 코드입력
m = folium.Map(location=[latitude, longitude],
               zoom_start=17, 
               width=750, 
               height=500
              )
m

## 마커 추가

- `location`: 마커를 추가할 **위도/경도 좌표**를 입력 후 
- `popup`: 표기할 팝업 문구 지정 (마우스 클릭시 표기되는 문구)
- `tooltip`: 표기할 툴팁 지정 (마우스 오버시 표기되는 문구)

마커를 생성 후 기존에 생성된 지도 `m`에 추가한다.

In [ ]:
# 코드입력
folium.Marker([latitude, longitude],
              popup="서울역",
              tooltip="1호선 4호선").add_to(m)
m

마커 스타일 변경

In [ ]:
# 코드입력
folium.Marker([latitude, longitude],
              popup="서울역",
              tooltip="1호선 4호선", 
              icon=folium.Icon('red', icon='star'),
             ).add_to(m)
m

In [ ]:
folium.CircleMarker([latitude, longitude],
                    color='tomato',
                    radius = 50, 
                    tooltip='서울역 상권').add_to(m)
m

### 데이터셋

[소상공인 시장 진흥공단 상권정보](https://www.data.go.kr/tcs/dss/selectFileDataDetailView.do?publicDataPk=15083033)

In [ ]:
import pandas as pd

# 코드입력
df = pd.read_csv('상권정보_서울.csv', encoding='cp949')
df.head(3)

컬럼 출력 

In [ ]:
df.columns

행정동명이 `청파동`, `남영동`, `소공동` 인 지역만 불러온다

In [ ]:
df[df['행정동명'].isin(['청파동', '남영동', '소공동'])]

In [ ]:
sub_df = df[df['행정동명'].isin(['청파동', '남영동', '소공동'])]
sub_df.head(3)

In [ ]:
df2=sub_df[['위도', '경도', '상호명','행정동명']]
df2

## 클러스터 생성

MarkerCluster 를 이용하면 가까운 거리의 Marker들을 군집시켜서 해당 건수를 표현해준다. 특정 클러스터를 클릭하면 해당 위치를 확대시켜 자세히 들여다볼 수 있다.

In [ ]:
m = folium.Map(
    location=[latitude, longitude], 
    zoom_start=15
)
m

In [ ]:
# 코드입력
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

for i in range(len(df2)):
    lat = df2.iloc[i,0]
    long = df2.iloc[i,1]
    folium.Marker([lat, long] ,
                  icon = folium.Icon(color="green")
                  ).add_to(marker_cluster)
m

In [ ]:
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

for lat, long in zip(df2['위도'], df2['경도']):
    folium.Marker([lat, long], 
                  icon = folium.Icon(color="green")
                 ).add_to(marker_cluster)
m

## 서울 지도에서 행정 구역별 표시

In [ ]:
# 서울의 행정구역 경계를 GeoJSON으로 표현한 파일 업로드 

geo_path = 'seoul_municipalities_geo_simple.json'

import json
seoul_geo = json.load(open(geo_path, encoding="utf-8"))
seoul_geo

서울 지역의 구별 boundary 시각화

In [ ]:
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11, 
)

# 코드입력
folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)

m

## tiles 옵션 변경을 통해 지도의 테마 변경

지도의 기본 테마(tiles)는 `OpenStreetMap`으로 설정되어 있는데, 이를 변경하여 다른 지도 테마를 적용할 수 있다.

### `Stamen Toner` 적용시

In [ ]:
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11, 
    # 타일변경
    tiles='Stamen Toner'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)

m

### `cartodbpositron` 적용시

In [ ]:
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11, 
    # 타일변경
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)

m

## 서울시 자치구별 상권정보 시각화

- [소상공인시장진흥공단 상권 정보 데이터셋 다운로드](https://www.data.go.kr/data/15083033/fileData.do)

In [ ]:
import pandas as pd

# 서울 상권정보 데이터를 로드합니다
seoul = pd.read_csv('상권정보_서울.csv', encoding='cp949')

# 필요한 컬럼 정보만 가져옵니다
seoul = seoul[['시군구명', '상권업종대분류명', '상권업종중분류명', '위도', '경도']]
seoul.head(10)

### 서울시의 업종별 개수 시각화

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정
plt.rcParams['font.family'] = 'NanumGothic'

서울시의 `상권업종중분류명`에 따른 데이터 개수

In [ ]:
seoul['상권업종중분류명'].value_counts()

In [ ]:
# 코드입력
plt.figure(figsize=(12, 30))
sns.countplot(y=seoul['상권업종중분류명'])
                          # order=seoul['상권업종중분류명'].value_counts().index

plt.yticks(fontsize=12)
plt.title('서울시 업종별 개수')
plt.show()

 '도서관/독서실' 업종은 자치구별로 어느 곳에 가장 많이 분포해 있는지 시각화

In [ ]:
seoul_lib = seoul[seoul['상권업종중분류명'] == '도서관/독서실']
seoul_lib

In [ ]:
# 서울의 행정구역 경계를 GeoJSON으로 표현한 파일 업로드 

geo_path = 'seoul_municipalities_geo_simple.json'

import json
seoul_geo = json.load(open(geo_path, encoding="utf-8"))
seoul_geo

In [ ]:
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=12, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)

클러스터를 추가하여 지도위에 시각화 

In [ ]:
# 코드입력
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(m)

for lat, long in zip(seoul_lib['위도'], seoul_lib['경도']):
    folium.Marker([lat, long] ,
                  icon = folium.Icon(color="green")
                  ).add_to(marker_cluster)
m

`choropleth` 를 사용하여 업종 별 개수에 따른 색상의 차이를 둔다.

In [ ]:
seoul_group_data = seoul_lib.groupby('시군구명')['상권업종중분류명'].count()
seoul_group_data

In [ ]:
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11, 
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)
m

In [ ]:
# 코드입력
m.choropleth(geo_data=seoul_geo,
             data=seoul_group_data, 
             fill_color='YlOrRd', # 색상 변경도 가능하다
             fill_opacity=0.5,
             line_opacity=0.2,
             key_on='properties.name',
             legend_name="지역구별 도서관/독서실 수"
            )
m

## Html 파일로 저장

In [ ]:
m.save('map.html')